## En este conjunto de Notebooks (1 al 9), se hace un Anális Exploratorio, Limpieza y Preprocesamiento detallado de un archivo excel que contiene datos de diferentes clientes de una compañía  de tarjetas de crédito, incluida la información de si después de 6 meses de datos históricos, ellos han fallado en el pago de la tarjeta ( pago mínimo).   El objetivo es construir un modelo de clasificación que prediga si un determinado cliente  pagará o no su tarjeta el mes siguiente.

#### Diccionario de datos


**LIMIT_BAL**: Cantidad de crédito otorgado en la tarjeta.

**SEX**: Género (1 = Hombre; 2 = Mujer).

**EDUCATION**: Educación (1 = Escolar graduado; 2 = universitaria; 3 = high school; 4 = otros).

**MARRIAGE**: Estado Marital (1 = casado; 2 = soltero; 3 = otros).

**AGE**: Edad (en años).

**PAY_1–PAY_6**: Pagos mensuales anteriores, desde Abril hasta Septiembre.


**PAY_1** Es el estado de pago en Septiembre; _PAY_2_ es el de Agosto y así hastao _PAY_6_,
que es el estado de pago en Abril..
La escala de medida es:-2=balance cero sin uso de tarjeta, -1 = pago debido; 0=Hecho el pago mínimo, 1 = pago atrasado por un mes; 2 = atraso de dos meses. y así hasta 8 = atraso de pago de 8 meses y más. -2=sin uso de tarjeta.

**BILL_AMT1–BILL_AMT6_**: Saldo de la cuenta (en dólares Taiwaneses).

**BILL_AMT1** Estado de cuenta en Septiembre; **BILL_AMT2** Estado de cuenta en Agosto;
hasta **BILL_AMT6**, estado de cuenta en Abril.

**PAY_AMT1–PAY_AMT6**: Pagos previos. PAY_AMT1 es la cantidad pagada en Septiembre;

**PAY_AMT2**  Agosto; hasta **PAY_AMT6**, cantidad pagada en Abril.  

**default payment next month**: Representa si se paga la tarjeta el mes siguiente. 1=pago, 0=no pago.

In [6]:
import pandas as pd

In [7]:
df=pd.read_excel('default_of_credit_card_clients_.xls')

In [8]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [9]:
df.shape

(30000, 25)

El archivo tiene 30000 registros(filas), cada uno representando un cliente (ID) y 25 columnas(features)

In [10]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

Veamos si efectivamente los ID son únicos.

In [12]:
df['ID'].nunique()

29687

Son 29687 distintos de los 30000. Así que vemos que Hay repeticiones de ID. 

####  DUPLICACIONES DE ID


In [13]:
id_counts=df['ID'].value_counts()

In [14]:
id_counts.head()

9e6e766d-ba75    2
96970f37-abaa    2
dab579ee-1645    2
c8bb762f-3ef9    2
84dc097a-dee3    2
Name: ID, dtype: int64

La columna  indica cada ID (5 primeros) y el número de veces que aparece en el archivo. Veamos ahora las distintas repeticiones:

In [15]:
id_counts.value_counts()

1    29374
2      313
Name: ID, dtype: int64

Esto indica que hay 313 clientes duplicados en el archivo.No hay repeticiones de más de dos. 29374 corresponden con valores únicos. Veremos la naturaleza de estas repeticiones e intentaremos eliminarlas. 

In [16]:
dup_ids=id_counts.index[id_counts==2]  #identifica los índices de los ID's duplicados.

In [17]:
dup_ids=list(dup_ids)

In [18]:
len(dup_ids)

313

corresponde con el número obtenido antes!
Veamos los primeros 3 pares de ID's duplicados:

In [19]:
df.loc[df['ID'].isin(dup_ids[0:3]),:]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
19080,dab579ee-1645,210000,2,1,2,29,0,0,0,0,...,198293,197841,205864,7700,10292,8900,7200,14300,10000,0
19180,dab579ee-1645,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24908,96970f37-abaa,210000,1,1,2,32,-2,-2,-2,-2,...,5365,789,1086,1146,3988,5381,791,1088,948,0
25008,96970f37-abaa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29450,9e6e766d-ba75,200000,1,1,2,30,1,-1,-1,-1,...,0,1275,543,1870,1000,0,1275,543,0,0
29550,9e6e766d-ba75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Hay filas de puros ceros en ID's duplicados. Esas son las que hay que borrar:

In [20]:
df_zero_mask= df == 0     #identifica las filas con puros ceros

In [21]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)

In [22]:
feature_zero_mask .head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [23]:
sum(feature_zero_mask)     #suma los true

315

Hay 315 filas con ceros. Son dos más de las con ID's duplicados. Las eliminaremos todas.

In [24]:
df_clean = df.loc[~feature_zero_mask,:].copy()

En el comando anterior, copiamos en un nuevo dataframe, todas las filas que no contienen ceros en todas sus columnas. Esto resuelve el problema de eliminar los ID's duplicados.

In [25]:
df_clean.shape

(29685, 25)

In [26]:
df_clean['ID'].nunique()

29685

Notemos que el valor está correcto, pues eran 29687 valores únicos, pero habían 2 filas más con identificadores únicos que tenían filas de puros ceros.

Guardaremos este Dataframe con nombre: df_clean_1.csv:

In [27]:
df_clean.to_csv('df_clean_1.csv',index=False)